## Generate Assignments from Survey Data

One common app used in integration with Workforce is Survey123. Dispatchers may want to take captured survey data to then drive new assignments - allowing the field worker to have control over what assignments are made. Say a field worker sees a sidewalk repair that needs to be completed, but doesn't have time to accomplish it or the dispatcher access to create an assignment. We can instead have them collect to a survey and then run our Jupyter Notebook to convert those surveys to assignments. 

In [61]:
import arcgis
from arcgis.apps import workforce
from arcgis.gis import GIS
from arcgis.geocoding import reverse_geocode
gis = GIS("https://arcgis.com", "workforce_scripts")
item = gis.content.get("c765482bd0b9479b9104368da54df90d")
project = workforce.Project(item)
project

Enter password: ········


<Project Dev Summit 2020: Configuring a Project>

### Get Survey Data

Let's see what surveys have been submitted to the survey "Sidewalk Repair" and visualize this data on the map.

In [58]:
survey_item = gis.content.get("9e97878a0d344408a5f2357d3e51aad4")
df = survey_item.layers[0].query('1=1',as_df=True)
df

,objectid,globalid,CreationDate,Creator,EditDate,Editor,notes,priority,new_assignment,SHAPE
0,1,00c245a4-3f29-4261-8613-ecdd30d92d1a,2020-02-21 16:26:13.954,workforce_scripts,2020-02-21 16:31:32.858,workforce_scripts,Asphalt,Low,Yes,"{""x"": -116.39117449518953, ""y"": 33.72203070875..."
1,2,ffb9205c-5039-46be-9f76-00980fedefb8,2020-02-21 16:26:52.822,workforce_scripts,2020-02-21 16:31:44.516,workforce_scripts,Gravel,Low,No,"{""x"": -116.24400582236498, ""y"": 33.76597659571..."
2,3,44812945-84d7-47e4-b037-40cd9c8daa6d,2020-02-21 16:28:07.827,workforce_scripts,2020-02-21 16:31:42.583,workforce_scripts,Concrete,Critical,Yes,"{""x"": -116.28632035178384, ""y"": 33.72907951333..."


In [59]:
# Visualize on map

webmap = gis.map("Palm Desert", zoomlevel=11)
webmap.basemap = 'streets-night-vector'
webmap.add_layer(survey_item.layers[0])
webmap

MapView(layout=Layout(height='400px', width='100%'), zoom=11.0)

### Get Surveys which should become new assignments

Now, let's take all the surveys which have "new_assignment" equal to Yes and create a corresponding assignment. We'll also need to ensure that we're not creating a duplicate - we'll do this by performing a check on the geometry of the assignments.


In [67]:
import math

def same_geometry(geometry_1, geometry_2):
    if math.isclose(geometry_1["x"], geometry_2["x"], abs_tol=0.001) and math.isclose(geometry_1["y"], geometry_2["y"], abs_tol=0.001):
        return True
    else:
        return False

assignments = project.assignments.search()
surveys = survey_item.layers[0].query('1=1').features
surveys_to_add = []
for survey in surveys:
    geometry = arcgis.geometry.project([survey.geometry], in_sr=4326,out_sr=3857)[0]
    if survey.attributes["new_assignment"] == "Yes" and not any(same_geometry(geometry, assignment.geometry) for assignment in assignments):
        surveys_to_add.append(survey)
        
print("Surveys to be added as Assignments: " + str(len(surveys_to_add)))

RequestException: {'code': 400, 'message': 'Unable to generate token', 'details': ['Unable to generate token for this server']}

### Create New Assignments and Visualize

Let's now add our surveys to the assignments and re-visualize them on a map. The neat thing here is we'll use reverse geocoding to store the location field!

In [66]:
new_assignments = []
for survey in surveys_to_add:
    new_assignments.append(
        workforce.Assignment(
            project,
            geometry=arcgis.geometry.project([survey.geometry], in_sr=4326,out_sr=3857)[0],
            location=reverse_geocode(survey.geometry)["address"]["Match_addr"],
            description=survey.attributes["notes"],
            priority=survey.attributes["priority"],
            assignment_type="Sidewalk Repair",
            status="unassigned"
        )
    )
project.assignments.batch_add(new_assignments)
webmap2 = gis.map("Palm Desert", zoomlevel=10)
webmap2.basemap = 'streets-night-vector'
webmap2.add_layer(survey_item.layers[0])
webmap2

[]